In [ ]:
from src.visualise import settings
from src.visualise.plot import plot_data
from src.data.paths import project_dir
from src.data.analysis import read_tiff_img, Circle, create_circular_mask
from src.data.detector import find_circle_hough_method, img_for_circle_detection

import numpy as np
import matplotlib.pyplot as plt
import scipy.ndimage as ndi
import re

from dataclasses import dataclass, field
from pathlib import Path

from copy import deepcopy

In [ ]:
@dataclass(frozen=True)
class DetectorImage:
    image: np.ndarray
    path: Path

    @property
    def init_circle(self) -> Circle:
        return Circle(x=self.image.shape[0]//2, y=self.image.shape[1]//2, r=100)

@dataclass(frozen=True)
class DetectorData:
    raw: DetectorImage
    lv: DetectorImage
    det_no: int

@dataclass(frozen=True)
class DetectorDataCollection:
    path: Path
    data: dict[int, DetectorData] = field(default_factory=dict)

    def __post_init__(self):
        if not self.data:
            self._load_data()

    def _load_data(self):
        for file_path in sorted(self.path.iterdir()):
            if file_path.name.endswith('lv'):
                # get detector data
                det_id = re.findall(r'\d+', file_path.name)[0]
                det_no = int(det_id)
                # live view images
                lv_path = next(file_path.glob('**/*tif'))
                lv_data = read_tiff_img(lv_path, border_px=0)
                lv_image = DetectorImage(image=lv_data, path=lv_path)
                # raw data images
                try:
                    raw_path = next((self.path / det_id).glob('**/*tif'))
                    raw_data = read_tiff_img(raw_path, border_px=0)
                    raw_image = DetectorImage(image=raw_data, path=raw_path)
                    det_data = DetectorData(raw=raw_image, lv=lv_image, det_no=det_no)
                    self.data[det_no] = det_data
                    print(f"{det_no} ", end='')
                except StopIteration:
                    print(f"missing_{det_no} ", end='')

In [ ]:
raw_path = project_dir / 'data' / 'raw' / '2024-02-20'
proton_data = DetectorDataCollection(path=raw_path)

In [ ]:
co60_path = project_dir / 'data' / 'raw' / 'Co60'
co60_data = DetectorDataCollection(path=co60_path)

In [ ]:
proton_data.data[1].lv.init_circle

# Proton raw data

In [ ]:
fig, ax = plot_data(proton_data.data[1].lv.image, path='', circle_px=proton_data.data[1].lv.init_circle, figsize=(10,12))

In [ ]:
fig, ax = plot_data(proton_data.data[1].raw.image, path='', circle_px=proton_data.data[1].raw.init_circle, figsize=(10,12))

# Co60 raw data

In [ ]:
fig, ax = plot_data(co60_data.data[1].lv.image, path='', circle_px=co60_data.data[1].lv.init_circle, figsize=(10,12))

In [ ]:
fig, ax = plot_data(co60_data.data[1].raw.image, path='', circle_px=co60_data.data[1].raw.init_circle, figsize=(10,12))

## Background

In [ ]:
background_path = next(raw_path.parent.glob('**/*background*/**/**/*tif'))
background_data = DetectorImage(path=background_path, image=read_tiff_img(background_path, border_px=0))

In [ ]:
background_data.image.max()

In [ ]:
fig, ax = plot_data(background_data.image, path='', circle_px=background_data.init_circle, figsize=(10,12))

# Background subtraction

In [ ]:
proton_bg_sub_data = DetectorDataCollection(path=proton_data.path, data=deepcopy(proton_data.data))
for data in proton_bg_sub_data.data.values():

    # out data are save as uint16, dataclasses are frozen
    # its not straightforward to use np.crop(0) or cast to int64
    # therefore we shift data up, perform background subtraction, crop negative values and shift back
    np.add(data.raw.image, background_data.image.max(), out=data.raw.image)    
    np.subtract(data.raw.image, background_data.image, out=data.raw.image)
    np.clip(data.raw.image, a_min=background_data.image.max(), a_max=None, out=data.raw.image)
    np.subtract(data.raw.image, background_data.image.max(), out=data.raw.image)


In [ ]:
co60_bg_sub_data = DetectorDataCollection(path=co60_data.path, data=deepcopy(co60_data.data))
for data in co60_bg_sub_data.data.values():

    # out data are save as uint16, dataclasses are frozen
    # its not straightforward to use np.crop(0) or cast to int64
    # therefore we shift data up, perform background subtraction, crop negative values and shift back
    np.add(data.raw.image, background_data.image.max(), out=data.raw.image)    
    np.subtract(data.raw.image, background_data.image, out=data.raw.image)
    np.clip(data.raw.image, a_min=background_data.image.max(), a_max=None, out=data.raw.image)
    np.subtract(data.raw.image, background_data.image.max(), out=data.raw.image)

In [ ]:
fig, ax = plot_data(proton_data.data[1].raw.image, path='', circle_px=proton_data.data[1].raw.init_circle, figsize=(10,12))

In [ ]:
fig, ax = plot_data(proton_bg_sub_data.data[1].raw.image, path='', circle_px=proton_bg_sub_data.data[1].raw.init_circle, figsize=(10,12))

## Detector discovery

In [ ]:
# import numpy.typing as npt
# import logging
# def img_for_circle_detection(data: npt.NDArray, r: float = 100, nsigma: float = 2., old_method : bool = False) -> npt.NDArray:
#     logging.info('Before setting threshold ' +
#                  f'min {np.nanmin(data)}, mean {np.nanmean(data):3.3f}, max {np.nanmax(data)}')
#     output = np.zeros(shape=data.shape, dtype='uint8')

#     circle_inside_det = Circle(x=data.shape[0]/2,y=data.shape[0]/2,r=r)
#     mask_inside_det = create_circular_mask(data, circle_inside_det)
#     upper_threshold = np.mean(data, where=mask_inside_det) + nsigma * np.std(data, where=mask_inside_det)
#     lower_threshold = np.mean(data, where=mask_inside_det) - nsigma * np.std(data, where=mask_inside_det)

#     if old_method:
#         lower_threshold = np.percentile(data, 95)
#         upper_threshold = np.max(data)
#     print(lower_threshold, upper_threshold)
    
#     output[(data > lower_threshold) & (data < upper_threshold)] = 255
#     output[np.isnan(data)] = 255
#     logging.info('After setting threshold ' +
#                  f'min {np.min(output)}, mean {np.mean(output):3.3f}, max {np.max(output)}')
#     return output

# import cv2
# def find_circle_hough_method(data: npt.NDArray) -> Circle:
#     logging.info(f'Detector circle not provided, calculating with Hough method')
#     hough_results = cv2.HoughCircles(data, cv2.HOUGH_GRADIENT, dp=1, minDist=10000, param1=10, param2=0.9, minRadius=100, maxRadius=200)
#     logging.info(f'Hough method results {hough_results}')
#     result_circle = Circle()
#     if hough_results is None:
#         print("No detector found by Hough method")
#     elif hough_results.shape[0] > 1:
#         print("More than one shape found by Hough method")
#     elif hough_results.shape[0] == 1 and hough_results.shape[1] == 1:
#         # explicit conversion to float is needed to ensure proper JSON serialisation
#         # hough_results is a numpy float32 array and float32 is not JSON serialisable
#         result_circle = Circle(
#             x=float(hough_results[0, 0, 0]),
#             y=float(hough_results[0, 0, 1]),
#             r=float(hough_results[0, 0, 2]),
#         )
#         logging.info(f'Detected circle {result_circle}')
#     return result_circle

In [ ]:
lv_for_detect = img_for_circle_detection(proton_bg_sub_data.data[1].lv.image, nsigma=2)

In [ ]:
find_circle_hough_method(lv_for_detect)

In [ ]:
fig, ax = plot_data(proton_data.data[1].lv.image, path='', circle_px=find_circle_hough_method(lv_for_detect), figsize=(10,12))

In [ ]:
fig, ax = plot_data(proton_data.data[1].raw.image, path='', circle_px=find_circle_hough_method(lv_for_detect), figsize=(10,12))

In [ ]:
co60_data_bg_removed = (co60_data.astype(np.int64)-background_data.astype(np.int64)).clip(0,None)
mask_for_circle = create_circular_mask(img=co60_data_bg_removed, circle_px=Circle(c.x, c.y, 80))
co60_data_bg_removed_mean = np.mean(co60_data_bg_removed[mask_for_circle], where=co60_data_bg_removed[mask_for_circle]>0)
co60_data_bg_removed_std = np.std(co60_data_bg_removed[mask_for_circle], where=co60_data_bg_removed[mask_for_circle]>0)
co60_data_bg_removed_mean, co60_data_bg_removed_std / co60_data_bg_removed_mean

In [ ]:
dose_Co60 = 5
co60_data_bg_removed_mean / dose_Co60

In [ ]:
plot_data((proton_data.astype(np.int64)-background_data.astype(np.int64)).clip(0,None), path='', circle_px=Circle(c.x, c.y, 80))

In [ ]:
dose_proton_Gy = 5

proton_data_bg_removed = (proton_data.astype(np.int64)-background_data.astype(np.int64)).clip(0,None)
mask_for_circle = create_circular_mask(img=proton_data_bg_removed, circle_px=Circle(c.x, c.y, 80))
proton_data_bg_removed_mean = np.mean(proton_data_bg_removed[mask_for_circle], where=proton_data_bg_removed[mask_for_circle]>0)
proton_data_bg_removed_std = np.std(proton_data_bg_removed[mask_for_circle], where=proton_data_bg_removed[mask_for_circle]>0)
proton_data_bg_removed_mean

proton_data_bg_removed_mean / dose_proton_Gy, proton_data_bg_removed_std / proton_data_bg_removed_mean

# Efficiency

In [ ]:
co60_signal_per_Gy = (co60_data.astype(np.int64)-background_data.astype(np.int64)).clip(1,None) / dose_Co60
proton_signal_per_Gy = (proton_data.astype(np.int64)-background_data.astype(np.int64)).clip(1,None) / dose_proton_Gy
plot_data((proton_signal_per_Gy / co60_signal_per_Gy).clip(0.001,2), path='', circle_px=Circle(c.x, c.y, 80))

In [ ]:
(proton_data_bg_removed_mean / dose_proton_Gy) / (co60_data_bg_removed_mean / dose_Co60)

In [ ]:
plot_data(ndi.median_filter((proton_signal_per_Gy / co60_signal_per_Gy).clip(0.001,2),size=20), path='', circle_px=Circle(c.x, c.y, 80))